# Examining GangSTR STR genotypes based on bowtie2 alignments
### BIO392 05.10.2022

Contact: Max Verbiest (maxadriaan.verbiest@uzh.ch)

In this notebook, we will explore whether the aligments we generated using bowtie2 are suitable to genotype STR lengths with.

### 1: Load libraries and files

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gangstr_utils

%matplotlib inline

In [2]:
df_str_loci = pd.read_csv(
    "../data/repeats/APC_repeats.tsv", 
    sep="\t", 
    header=None,
    names=["chr", "start", "end", "unit_len", "unit"])

df_gangstr_mut = gangstr_utils.load_gangstr_output(df_str_loci, "../results/mutated/mutated.vcf")
df_gangstr_wt = gangstr_utils.load_gangstr_output(df_str_loci, "../results/wildtype/wildtype.vcf")

df_gangstr_merged = (gangstr_utils
                         .merge_samples(df_gangstr_mut, df_gangstr_wt)
                         .rename(columns={"alt_s1": "alt_mut", "alt_s2": "alt_wt"}))

### 2: Look for loci where the samples differ in STR copy number

In [3]:
df_gangstr_merged[df_gangstr_merged["sample_difference"] != 0]

,chr,start,end,unit_len,unit,ref,alt_mut,alt_wt,sample_difference
12,chr5,21005,21014,2,AT,5.0,2.0,5.0,3.0
68,chr5,117685,117718,2,AC,17.0,15.0,14.0,-1.0
86,chr5,137481,137490,2,AG,5.0,NaN,2.0,NaN


Strange! From the analyses we performed last week, we know that there should only be a difference in STR length between these two samples at one locus (the AG repeat at chr5:137481-137490). We now see differences at two other loci too. Furthermore, we were not even able to get an accurate estimate of STR length for chr5:137481-137490 in the mutated sample (you can look in the GangSTR vcf file for this sample to see what's going on). 

At every STR locus except for chr5:137481-137490, the samples should match the reference sequence (because I generated the inputs reads we used directly from the reference sequence). However, if we check the dataframes for STRs where the sample length does not match the reference length, we get many results:

In [4]:
df_gangstr_wt[df_gangstr_wt["ref"] != df_gangstr_wt["alt"]]

,chr,start,end,unit_len,unit,ref,alt
2,chr5,9390,9399,1,A,10.0,7.0
18,chr5,32477,32485,1,T,9.0,6.0
21,chr5,41515,41523,1,T,9.0,6.0
23,chr5,47414,47426,1,T,13.0,12.0
30,chr5,58315,58328,1,T,14.0,11.0
34,chr5,61748,61765,1,A,18.0,15.0
39,chr5,70019,70036,1,T,18.0,15.0
42,chr5,74772,74785,2,AC,7.0,4.0
45,chr5,76053,76066,1,T,14.0,13.0
50,chr5,85497,85508,4,CATA,3.0,1.0


In [5]:
df_gangstr_mut[df_gangstr_mut["ref"] != df_gangstr_mut["alt"]]

,chr,start,end,unit_len,unit,ref,alt
2,chr5,9390,9399,1,A,10.0,7.0
12,chr5,21005,21014,2,AT,5.0,2.0
18,chr5,32477,32485,1,T,9.0,6.0
21,chr5,41515,41523,1,T,9.0,6.0
23,chr5,47414,47426,1,T,13.0,12.0
30,chr5,58315,58328,1,T,14.0,11.0
34,chr5,61748,61765,1,A,18.0,15.0
39,chr5,70019,70036,1,T,18.0,15.0
42,chr5,74772,74785,2,AC,7.0,4.0
45,chr5,76053,76066,1,T,14.0,13.0


### 3: What is going on??

So, obviously, something is not quite right. Your task is now to find out where the observed errors come from, and how to fix it. The only tip you'll get for now is that there are problematic alignments in the BAM files we used, and that we want to extract only the 'properly paired' alignments. Luckily, the [samtools](https://www.htslib.org/doc/samtools.html) suite of tools (included in the BIO392_STRs conda env) offers all the functionality needed to address the problem. Particularly [samtools flagstat](https://www.htslib.org/doc/samtools-flagstat.html), [samtools view](https://www.htslib.org/doc/samtools-view.html) and [samtools flags](https://www.htslib.org/doc/samtools-flags.html) will be very useful. Going through these manual pages would be a good start. You can also visually inspect the aligments with [samtools tview](https://www.htslib.org/doc/samtools-tview.html).


**Task 1:** Find a way to identify problematic alignments in the BAM files generated by bowtie2. Extract only 'proper pairs', and save these in new BAM files.

**Task 2:** Rerun GangSTR using these new BAM files (you will need to change some things in the GangSTR bash script), and inspect the results using the code blocks at the start of this notebook.

**Task 3:** Were you able to solve/improve the outcome? Is bowtie2 a good aligner to use for genotyping STRs?

# task 1: 
samtools flagstat /Users/lschneid/2022-10-05/data/alignments/mutated/APC_mut.bam: 23944 + 0 properly paired (49.97% : N/A)    
samtools flagstat /Users/lschneid/2022-10-05/data/alignments/wildtype/APC_wt.bam: 23980 + 0 properly paired (50.05% : N/A)    
use samtools tview -bf 0x2 to filter for the proper pairs

In [20]:
# task 2
df_str_loci = pd.read_csv(
    "../data/repeats/APC_repeats.tsv", 
    sep="\t", 
    header=None,
    names=["chr", "start", "end", "unit_len", "unit"])

df_gangstr_mut = gangstr_utils.load_gangstr_output(df_str_loci, "../results/mutated/mutated.vcf")
df_gangstr_wt = gangstr_utils.load_gangstr_output(df_str_loci, "../results/wildtype/wildtype.vcf")

df_gangstr_merged = (gangstr_utils
                         .merge_samples(df_gangstr_mut, df_gangstr_wt)
                         .rename(columns={"alt_s1": "alt_mut", "alt_s2": "alt_wt"}))

In [21]:
df_gangstr_merged[df_gangstr_merged["sample_difference"] != 0]

,chr,start,end,unit_len,unit,ref,alt_mut,alt_wt,sample_difference
86,chr5,137481,137490,2,AG,5.0,NaN,5.0,NaN


In [24]:
df_gangstr_wt[df_gangstr_wt["ref"] != df_gangstr_wt["alt"]]

,chr,start,end,unit_len,unit,ref,alt


In [25]:
df_gangstr_mut[df_gangstr_mut["ref"] != df_gangstr_mut["alt"]]

,chr,start,end,unit_len,unit,ref,alt
86,chr5,137481,137490,2,AG,5.0,NaN


# task 3: 
Were you able to solve/improve the outcome?     
* yes, we were able to improve the outcome.    

Is bowtie2 a good aligner to use for genotyping STRs?     
* bowtie2 is a good aligner for this task, we were able to do it fast 